In [1]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.wkt import loads
from shapely.geometry import Point, LineString

In [2]:
health_facilities = gpd.read_file('../data/external/healthsitesio.geojson')
health_facilities = health_facilities[['osm_id', 'amenity', 'name', 'geometry']]
health_facilities['geometry'] = health_facilities['geometry'].centroid
health_facilities["longitude"] = health_facilities["geometry"].apply(lambda p: p.x)
health_facilities["latitude"] = health_facilities["geometry"].apply(lambda p: p.y)

df = pd.read_excel("../data/raw/Central Mozambique - Multi-Sectorial Location Assessment Dataset - Round 20_0.xlsx", sheet_name='Sheet1')[['SSID', '1.1.d.1 Site Name', 'Longitude', 'Latitude', 'Site status', '2.1.b.1 Total number of IDP individuals', 'How long does it take to reach the nearest health facility from the site?']]
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
sites = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
sites["longitude"] = sites["geometry"].apply(lambda p: p.x)
sites["latitude"] = sites["geometry"].apply(lambda p: p.y)



/var/folders/q2/r6cyvbjx443d2txp6kr8tl_h0000gn/T/ipykernel_24553/3039576180.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  health_facilities['geometry'] = health_facilities['geometry'].centroid


In [3]:

# clip to cmaller area
bounding_area = gpd.read_file("bounding_box_large.geojson")
health_facilities = gpd.clip(health_facilities, bounding_area)
sites = gpd.clip(sites, bounding_area)
bbox = bounding_area.total_bounds


In [4]:
graph = ox.load_graphml(filepath="../data/external/moz.graphml")
graph = ox.truncate.truncate_graph_bbox(graph, bbox=bbox)
#graph = ox.simplification.simplify_graph(graph)

In [5]:
# Find nearest network nodes for sites and health facilities
sites["nearest_node"] = sites.apply(lambda row: ox.distance.nearest_nodes(graph, row["longitude"], row["latitude"]), axis=1)
health_facilities["nearest_node"] = health_facilities.apply(lambda row: ox.distance.nearest_nodes(graph, row["longitude"], row["latitude"]), axis=1)

In [6]:
# Compute shortest path to the nearest health facility
def find_nearest_facility(site_node, facility_nodes, graph, health_facilities):
    shortest_distance = float("inf")
    nearest_facility = None
    nearest_facility_name = None
    shortest_path = []
    for facility_node in facility_nodes:
        try:
            path_length = nx.shortest_path_length(graph, site_node, facility_node, weight="length")
            if path_length < shortest_distance:
                shortest_distance = path_length
                nearest_facility = facility_node
                nearest_facility_name = health_facilities.loc[health_facilities["nearest_node"] == facility_node, "name"].values[0]
                shortest_path = nx.shortest_path(graph, site_node, facility_node, weight="length")
        except nx.NetworkXNoPath:
            continue
    return nearest_facility, nearest_facility_name, shortest_distance, shortest_path

facility_nodes = health_facilities["nearest_node"].tolist()
sites[["nearest_health_facility_node", "nearest_health_facility_name", "distance_meters", "route"]] = sites.apply(
    lambda row: find_nearest_facility(row["nearest_node"], facility_nodes, graph, health_facilities),
    axis=1, result_type="expand"
)
sites["walking_time_minutes"]= sites["distance_meters"]/83.4



In [7]:

# Save results
sites.to_csv("temp3.csv", index=False)
sites.to_parquet('data3.parquet') 

In [8]:
sites

,SSID,1.1.d.1 Site Name,Longitude,Latitude,Site status,2.1.b.1 Total number of IDP individuals,How long does it take to reach the nearest health facility from the site?,geometry,longitude,latitude,nearest_node,nearest_health_facility_node,nearest_health_facility_name,distance_meters,route,walking_time_minutes
9,MOZ_R013,Machacuari,33.301000,-20.105000,Open,115,More than 60 minutes,POINT (33.301 -20.105),33.301000,-20.105000,6403355793,6441867825,Posto de Saude da Missao,17785.851189,"[6403355793, 6403355481, 6403356063, 640335605...",213.259607
25,MOZ_R038,Muchambanha,33.323000,-20.083000,Open,214,More than 60 minutes,POINT (33.323 -20.083),33.323000,-20.083000,6403360957,6441867825,Posto de Saude da Missao,20292.681516,"[6403360957, 6403360953, 6403360984, 654975005...",243.317524
48,MOZ_R065,Nhamississua,33.307000,-20.054000,Open,182,More than 60 minutes,POINT (33.307 -20.054),33.307000,-20.054000,6402220991,6441867825,Posto de Saude da Missao,13429.136848,"[6402220991, 6402221339, 6402221334, 640222142...",161.020826
40,MOZ_R054,Gudza,33.335000,-20.039000,Open,864,More than 60 minutes,POINT (33.335 -20.039),33.335000,-20.039000,6402282789,6441867825,Posto de Saude da Missao,14608.555381,"[6402282789, 587735325, 6402240775, 6402282876...",175.162535
12,MOZ_R020,Javera,33.332000,-20.020000,Open,134,31 - 60 minutes,POINT (33.332 -20.02),33.332000,-20.020000,6400301453,6441867825,Posto de Saude da Missao,8878.354648,"[6400301453, 6402499818, 6547962028, 640117872...",106.455092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,MOZ_R056,Mucoa,37.180967,-17.473090,Open,426,31 - 60 minutes,POINT (37.18097 -17.47309),37.180967,-17.473090,2508915261,2508915261,Centro de Saude Furquia,0.000000,[2508915261],0.000000
42,MOZ_R057,Munguissa,37.201818,-17.457714,Open,863,31 - 60 minutes,POINT (37.20182 -17.45771),37.201818,-17.457714,3947853102,4090884319,Centro de Saude de Maquival-Rio,76009.371639,"[3947853102, 3947853185, 4128227477, 250890626...",911.383353
19,MOZ_R032,Mussaia,37.353718,-17.395727,Open,3305,16 - 30 minutes,POINT (37.35372 -17.39573),37.353718,-17.395727,2509169059,4494322019,Centro de Saude de Mabala,66823.805360,"[2509169059, 2509169625, 2509170000, 250917002...",801.244669
58,MOZ_R078,Landinho,37.518433,-17.285767,Open,1500,Less than 15 minutes,POINT (37.51843 -17.28577),37.518433,-17.285767,4489819726,2522209875,Centro de Saude de Mugaua,12614.130604,"[4489819726, 4489819718, 4489819708, 925662504...",151.248568


In [9]:
ox.io.save_graphml(graph, filepath="../data/processed/moz.graphml")

In [10]:
sites.to_parquet('../data/processed/moz.parquet') 